1. Retrieve table names from the database (telecom.db).

2. Join all tables in the database on the primary key.

3. Find unique occurrences of event_type and severity using an SQL query.

4. Find how many occurrences there are of each fault_severity using an SQL query.